In [1]:
from utils import prepare_jupyter
prepare_jupyter()

In [14]:
import numpy as np
import networkx as nx

from IPython.display import Image
from structure import RandomForestEnsemble, Dataset

train_set, val_set = Dataset.create_iris().split(0.2)

params = {
    'n_estimators': 1000,
    'max_depth': 10
}

rf = RandomForestEnsemble(params)
rf.fit(train_set)

tree_a, tree_b = rf.trees[0], rf.trees[4]

In [ ]:
from itertools import product

# nx_a = tree_a.tree
# nx_b = tree_b.tree

def compare_nodes(node_a, node_b):
    if node_a['is_leaf'] and node_b['is_leaf']:
        return node_a['target'] == node_b['target']
    elif not node_a['is_leaf'] and not node_b['is_leaf']:
        return node_a['threshold'] == node_b['threshold'] and node_a['feature'] == node_b['feature']
    
    return False

results = []
for tree_a, tree_b in product(rf.trees, rf.trees):
    if tree_a == tree_b:
        continue
        
    result = nx.graph_edit_distance(tree_a.tree, tree_b.tree, node_match=compare_nodes)
    results.append(results)
    
results = np.array(results)
# nx.graph_edit_distance(nx_a, nx_b, node_match=compare_nodes)

In [9]:
tree_a.used_attrs, tree_b.used_attrs

({'1__2.649999976158142__$<=',
  '1__2.75__$<=',
  '1__3.049999952316284__$<=',
  '2__2.599999964237213__$<=',
  '2__4.950000047683716__$<=',
  '2__5.049999952316284__$<=',
  '3__1.649999976158142__$<=',
  '3__1.75__$<='},
 {'2__2.449999988079071__$<=',
  '2__4.950000047683716__$<=',
  '2__5.450000047683716__$<=',
  '3__1.550000011920929__$<=',
  '3__1.6500000357627869__$<='})

In [15]:
rf.used_attr_diversity(), rf.used_feature_diversity()

(0.12866870749762946, 0.16839220405898042)